In [1]:
import sys
from pathlib import Path

parent = str(Path().resolve().parent)
if parent not in sys.path:
    sys.path.insert(0, parent)

In [2]:
#download python3-discogs-client
import discogs_client
import pandas as pd
import ast
import numpy as np

In [3]:
consumer_key = 'ykUodTXeeVzqTJkACRzX'
consumer_secret = 'fOWGGoeHRESxkDPVoPFQKJBaTtDrFFRn'

d = discogs_client.Client('CSS_memberfetch/1.0', consumer_key=consumer_key, consumer_secret=consumer_secret)

# Generate the authorize URL
request_token, request_token_secret, authorize_url = d.get_authorize_url()

In [4]:
authorize_url

'https://www.discogs.com/oauth/authorize?oauth_token=COcpVsMuHmhciYYJLKvKtPpiJwMaVVDLzHyKHpnE'

In [5]:
# After authorization, Discogs will give you a verifier code
verifier = 'pEbSxZwxhr'
# Get the access token
access_token, access_token_secret = d.get_access_token(verifier)
#lad vær med at køre det her mere end 1 gang, så får man fejl.

In [6]:
d.set_token(access_token, access_token_secret)

In [7]:
def get_discogs_members(band_name):
    
    if band_name[-7:]==" (band)": # to avoid (band) ending for messing up search results
        band_name = band_name[:-7]
    
    results = d.search(band_name, type='artist') 
    if len(results)>0:
        i = 0
        while i < len(results):
            artist = results[i]
            if artist.name.lower() in band_name.lower():
                break
            i += 1
        
        if i != len(results):
            try:
                if hasattr(artist, 'members'):
                    return [member.name for member in artist.members]
            except discogs_client.exceptions.HTTPError as e:
                pass
    return []
    

In [8]:
# problem: nogle gange står navnene i profile afsnit

In [9]:
df_groups = pd.read_csv("../data/wip/groups_wMembers.csv")
df_discgroups = pd.read_csv("../data/wip/groups_wMembers2.csv")
df_groups = df_groups.drop("member", axis = 1)

### New fetch of the members where wiki-scraping only returned 1 member

In [10]:
df_discgroups["disc_members"] = [[] for _ in range(len(df_discgroups))]

In [11]:
df_discgroups

,pageid,title,cleaned_member,disc_members
0,489818,A Friend in London,['Tim Schou'],[]
1,157530,Ache,"['Per Wium', 'Peter Mellin', 'Finn Olafsson', ...",[]
2,255559,ADS (band),"['Michael Funder Thorlasius', 'Stig Pedersen',...",[]
3,976777,Agathorn,[],[]
4,493993,Alabama Black Snakes,"['Jonas Jessen', 'Hank Lee Tuffer', 'Mads Øste...",[]
...,...,...,...,...
1029,37360,Odense Symfoniorkester,"['Bjarne Hansen', 'Ulrike Kipp', 'Michaela Fuk...",[]
1030,162784,Sjællands Symfoniorkester,"['Ole Böhn', 'Harry Gram-Jensen', 'Fritz O. Ja...",[]
1031,10733,Sønderjyllands Symfoniorkester,"['Anne Augustinsson', 'Jesper Hindø', 'Nikolai...",[]
1032,369172,Aalborg Symfoniorkester,"['Marius Ungureanu', 'Ib Dirkov', 'Moshe Atzmo...",[]


In [12]:
# getting indexs of relevant rows
missing_members = df_discgroups[df_discgroups["cleaned_member"].apply(lambda x: len(ast.literal_eval(x))<=1)].index
wiki_indices = df_groups[df_groups["cleaned_member"].apply(lambda x: x != "[]")].index
members_to_check = [i for i in missing_members if i in wiki_indices]

In [13]:
df_discgroups.loc[members_to_check, 'disc_members'] = df_discgroups.loc[members_to_check, 'title'].apply(get_discogs_members)

HTTPError: 429: You are making requests too quickly.

In [148]:
df_discgroups["all_members"] = df_discgroups.apply(lambda row: np.unique(ast.literal_eval(row["cleaned_member"]) + (row["disc_members"])), axis = 1)

In [165]:
df_discgroups["all_members"] = df_discgroups["all_members"].apply(lambda memberlist: [str(member) for member in memberlist])

In [166]:
df_discgroups

,pageid,title,cleaned_member,disc_members,all_members
0,489818,A Friend in London,['Tim Schou'],"[Tim Schou, Esben Svane, Aske Damm Bramming, S...","[Aske Damm Bramming, Esben Svane, Sebastian Vi..."
1,157530,Ache,"['Per Wium', 'Peter Mellin', 'Finn Olafsson', ...",[],"[Finn Olafsson, Per Wium, Peter Mellin, Steen ..."
2,255559,ADS (band),"['Michael Funder Thorlasius', 'Stig Pedersen',...",[],"[Karsten Hjarsø, Lars Top-Galia, Martin Krogh,..."
3,976777,Agathorn,[],[],[]
4,493993,Alabama Black Snakes,"['Jonas Jessen', 'Hank Lee Tuffer', 'Mads Øste...",[],"[Gustav Pontoppidan, Hank Lee Tuffer, Jonas Je..."
...,...,...,...,...,...
1029,37360,Odense Symfoniorkester,"['Bjarne Hansen', 'Ulrike Kipp', 'Michaela Fuk...",[],"[Alf Vestergaard, Anca Bold, Angelika Wagner, ..."
1030,162784,Sjællands Symfoniorkester,"['Ole Böhn', 'Harry Gram-Jensen', 'Fritz O. Ja...",[],"[Fritz O. Jacobsen, Harry Gram-Jensen, Ilse O...."
1031,10733,Sønderjyllands Symfoniorkester,"['Anne Augustinsson', 'Jesper Hindø', 'Nikolai...",[],"[Anne Augustinsson, Brian Quist Jørgensen, Hen..."
1032,369172,Aalborg Symfoniorkester,"['Marius Ungureanu', 'Ib Dirkov', 'Moshe Atzmo...",[],"[Bettina Ejlerts Jensen, Bo Juel Christiansen,..."


In [ ]:
#df_discgroups.to_csv("data/groups_wMembers3.csv", index=False)
df_discgroups

### Original fetch of discogs members 
(the ones who didnt have any results from wikipedia)

In [ ]:
empty_member_indices = df_groups[df_groups["cleaned_member"].apply(lambda x: x == "[]")].index
df_groups.loc[empty_member_indices, 'cleaned_member']= df_groups.loc[empty_member_indices, 'title'].apply(get_discogs_members)

In [ ]:
# df_groups.to_csv("data/groups_wMembers2.csv", index=False)

In [ ]:
# problem: nogle gange er der flere bands med samme navn på discogs (fx the fashion). 
# hvis ikke det danske band er det først, så er det svært at gøre noget ved det

In [ ]:
### problem: discogs har et tal efter personer hvis der er flere med samme navn
# ikke et problemm internt, men hvis det sakl sammenlignes med wiki går det galt
# måske kan vi bare fjerne det og håbe der ikke er flere med samme navn der optræder.